<font size = 5>这里是用残差来做<br></font>
<font size = 3>说实话，我一直不是很理解，为什么这个东西叫残差。在知道目标函数之前这个东西只是正在训练的模型，只是说这个模型可以和当前预测函数不同，难道就是应该这个差异，他才叫残差嘛<br></font>
<font size = 3></font>

In [1]:
import torch
import torch.nn as nn
from PIL import Image
import pandas as pd
from torchvision import models, transforms
import time
import numpy as np
import xml.etree.ElementTree as ET
import os
from torch.utils.data import Dataset

In [2]:
class MyDataset(Dataset):
    # 这里用transform参数是因为图片的像素是不同的，这里要对图片进行预处理
    def __init__(self, img_folder, label_folder, transform=None):
        super(MyDataset, self).__init__()
        self.map = {'boat':0,
                'dog':1,
                'bicycle':2,
                'person':3,
                'chair':4,
                'horse':5,
                'car':6,
                'aeroplane':7,
                'bird':8,
                'motorbike':9,
                'cow':10,
                'sheep':11,
                'bottle':12,
                'tvmonitor':13,
                'bus':14,
                'diningtable':15,
                'pottedplant':16,
                'cat':17,
                'train':18,
                'sofa':19}
        self.img_folder = img_folder
        self.label_folder = label_folder
        self.transform = transform
        self.image_files = sorted(os.listdir(img_folder))  # 获取图像文件列表
        self.label_files = sorted(os.listdir(label_folder))  # 获取标签文件列表

    def __len__(self):
        return len(self.image_files)

    # 这个函数还挺重要的，后面迭代器会使用
    def __getitem__(self, index):
        # 获取图像文件路径和标签文件路径
        img_name = self.image_files[index]
        # 因为label和img的区别只在其拓展名
        label_name = img_name.replace('.jpg', '.xml')
        img_path = os.path.join(self.img_folder, img_name)
        label_path = os.path.join(self.label_folder, label_name)

        # 读取图像和标签文件
        # 使用rgb格式输入
        image = Image.open(img_path).convert('RGB')

        # 这里读取标签的数据
        tree = ET.parse(label_path)
        root = tree.getroot()
        labels = []
        for obj in root.findall('object'):
            label = self.map.get(obj.find('name').text)
            labels.append(label)

        # 应用变换（如果有），就是调用传入的transform
        if self.transform is not None:
            image = self.transform(image)

        return image, label

# 示例用法

train_image_path = "C:\\Users\\30102\\Code\\Python\\DL\\CNN\\VOC\\train_JPEGImages"
train_label_path = "C:\\Users\\30102\\Code\\Python\\DL\\CNN\\VOC\\train_Annotations"
test_image_path = "C:\\Users\\30102\\Code\\Python\\DL\\CNN\\VOC\\test_JPEGImages"
test_label_path = "C:\\Users\\30102\\Code\\Python\\DL\\CNN\\VOC\\test_Annotations"


transform = transforms.Compose([
    # 调整图片给的size
    transforms.Resize((360, 360)),
    # 将图片给转成张量
    transforms.ToTensor(),
    # 标准化，为了提高准确度和更快收敛，我也不是很懂，反正人家写了，我就抄
    transforms.Normalize((0.5,), (0.5,))
])

# 创建自定义数据集实例
# 如果这里transform 写的是None的话，输出的是图片，但是这里转成张量了，所以读出的是张量
train_dataset = MyDataset(train_image_path, train_label_path, transform=transform)
test_dataset = MyDataset(test_image_path,test_label_path,transform=transform)



In [3]:
from torch.utils.data.sampler import SubsetRandomSampler

# 每一批50 个数据，我乱写的
batch_size = 20

# 使用处理器的个数
num_workers = 0

# 验证集所占的比例
valid_size = 0.2

# 打乱顺序
num_train = len(train_dataset)
indices = list(range(num_train))
np.random.shuffle(indices)
# print(indices)

# 训练集切片成训练集和验证集
split = int(np.floor(valid_size*num_train))
train_idx,valid_idx = indices[split:],indices[:split]

# 创建采样器
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size,
                                           sampler=train_sampler,num_workers = num_workers,pin_memory = True)
valid_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,
                                           sampler = valid_sampler,pin_memory = True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,num_workers=num_workers,pin_memory = True)

# 看看自己的data_loader 是不是有问题
for images, labels in train_loader:
    # 打印每个批次中的图像和标签
    for image, label in zip(images, labels):
        # 可以打印图像和标签的形状
        print("Image shape:", image.shape)
        print("Label:", label)

    # 一般来说，我们只需要打印或可视化第一个批次的数据
    break



Image shape: torch.Size([3, 360, 360])
Label: tensor(4)
Image shape: torch.Size([3, 360, 360])
Label: tensor(1)
Image shape: torch.Size([3, 360, 360])
Label: tensor(10)
Image shape: torch.Size([3, 360, 360])
Label: tensor(3)
Image shape: torch.Size([3, 360, 360])
Label: tensor(3)
Image shape: torch.Size([3, 360, 360])
Label: tensor(4)
Image shape: torch.Size([3, 360, 360])
Label: tensor(3)
Image shape: torch.Size([3, 360, 360])
Label: tensor(3)
Image shape: torch.Size([3, 360, 360])
Label: tensor(8)
Image shape: torch.Size([3, 360, 360])
Label: tensor(15)
Image shape: torch.Size([3, 360, 360])
Label: tensor(4)
Image shape: torch.Size([3, 360, 360])
Label: tensor(2)
Image shape: torch.Size([3, 360, 360])
Label: tensor(9)
Image shape: torch.Size([3, 360, 360])
Label: tensor(16)
Image shape: torch.Size([3, 360, 360])
Label: tensor(3)
Image shape: torch.Size([3, 360, 360])
Label: tensor(3)
Image shape: torch.Size([3, 360, 360])
Label: tensor(4)
Image shape: torch.Size([3, 360, 360])
Label:

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        # 这里是残差块
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=3, kernel_size=3, stride=1, padding=1)
        # 既然是直接学习残差了，我就省去池化的操作了，我也不知道为什么要池化
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 45 * 45, 4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096, 20)  # 假设输出类别数为 20
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # 前向传播过程，定义了输入数据如何通过网络层进行计算
        y = nn.functional.relu(self.conv1(x))
        y = nn.functional.relu(self.conv2(y))
        y = nn.functional.relu(self.conv3(y))
        # 这里进行了两次池化，每一次都缩小了，所以360/2 然后再 /2 得到90
        # 根据修改后的 self.fc1 的输入大小重新计算展平后的张量的大小
        x = x.view(-1, 64 * 45 * 45)  # 将多维张量展平为一维
        # 第一层全连接
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        # 第二层全连接
        x = self.fc2(x)
        x = self.dropout(x)
        # 第三层全连接
        x = self.fc3(x)
        return x

model = MyCNN().to('cuda')
print(model)